# COGS 118A Final Project
Malachi Mabie Dec 16 2020

### Dataset 1: Wine Quality
Citation:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.
http://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [3]:
from google.colab import drive
drive.mount('/content/drive')
whitewine_path = "/content/drive/My Drive/Colab Notebooks/COGS 118A/final project data/winequality-white.csv"
redwine_path = "/content/drive/My Drive/Colab Notebooks/COGS 118A/final project data/winequality-red.csv"
# data format:
"""Input variables (based on physicochemical tests):
   1 - fixed acidity
   2 - volatile acidity
   3 - citric acid
   4 - residual sugar
   5 - chlorides
   6 - free sulfur dioxide
   7 - total sulfur dioxide
   8 - density
   9 - pH
   10 - sulphates
   11 - alcohol
   Output variable (based on sensory data): 
   12 - quality (score between 0 and 10)"""
# we will convert the quality score to unacceptable / acceptable classification.

Mounted at /content/drive


'Input variables (based on physicochemical tests):\n   1 - fixed acidity\n   2 - volatile acidity\n   3 - citric acid\n   4 - residual sugar\n   5 - chlorides\n   6 - free sulfur dioxide\n   7 - total sulfur dioxide\n   8 - density\n   9 - pH\n   10 - sulphates\n   11 - alcohol\n   Output variable (based on sensory data): \n   12 - quality (score between 0 and 10)'

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()
sns.set_theme(style="ticks")

In [8]:
# load data
whitewine = pd.read_csv(whitewine_path, sep=';')
redwine = pd.read_csv(redwine_path, sep = ';')
whitewine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
# EDA - white
sns.pairplot(whitewine, hue="quality")

Output hidden; open in https://colab.research.google.com to view.

From this simple EDA, I've decided to ignore all "5" rated wines, since their mediocrity seems to gradient between every variable. I don't think I could place it as poor or good!

I also am not going to focuse on positive outliers, and will group them with acceptably good wines.

So, the usefulness of this classification will be to predict and prevent bad wines from happening! Ratings of 5 are like a margin of safety.

- 5:                0 (drop)
- 4,3 (& below):   -1
- 6,7 (& above):   +1

In [ ]:
whitewine.loc[whitewine['quality'] == 4].size / whitewine.size

0.03327888934258881

In [ ]:
whitewine.loc[whitewine['quality'] == 5].size / whitewine.size

0.2974683544303797

In [ ]:
whitewine.loc[whitewine['quality'] == 6].size / whitewine.size

0.44875459371171905

In [ ]:
whitewine.loc[whitewine['quality'] == 7].size / whitewine.size

0.17966516945692118

In [9]:
df = whitewine
df.loc[df['quality'] < 5, ['quality']] = -1
df.loc[df['quality'] == 5,['quality']] = 1    #UPDATE: for the Random Forest, I'll ONLY care about REALLY BAD wine.
df.loc[df['quality'] > 5, ['quality']] = 1

In [10]:
# see what the data looks like now
df.loc[df['quality'] == -1].size / df.size
# num 0s = 1457
# num 1s = 3258
# num -1s = 183

0.03736218864842793

In [ ]:
# drop our mediocre 0s
df.drop(df.loc[df['quality']==0].index, inplace=True)

In [11]:
# separate X and y
y = df['quality']
X = df.drop('quality',axis=1)

%store y X
%store y > y.txt
%store X > X.txt

Stored 'y' (Series)
Writing 'y' (Series) to file 'y.txt'.
Writing 'X' (DataFrame) to file 'X.txt'.


In [ ]:
%store -r

Note that our classes are inbalanced, so we should incorporate a classification trainer tuned for outlier detection.

## Classification Pipeline
  - 3 datasets
  - 3 algorithms
  - 3 trials per combo (27 total)
  - Compare accuracy

#### Dataset choices:
- wine quality (completed)
- chess wins subset (projected)
- swarm behavior (projected)

#### Algorithm choices:
- Support Vector Machine
  - Graph search through wide scale of hyperparameters
- Logistic Regression
  - Multinomial, newton-cg
- Random Forest
  - Won't use Isolation Forest bc the other algs aren't specialized for outliers


#### K-fold Cross Validation
1. Shuffle the dataset randomly.
2. Split the dataset into k groups
3. For each unique group:
  1. Take the group as a hold out or test data set
  2. Take the remaining groups as a training data set
  3. Fit a model on the training set and evaluate it on the test set
  4. Retain the evaluation score and discard the model
4. Summarize the skill of the model using the sample of model evaluation scores

In [ ]:
from sklearn.model_selection import StratifiedKFold
# stratified k-fold cross validation generator
k_fold = StratifiedKFold(n_splits=5, shuffle=True)

**Pipelines** let you combine all your feature engineering / pre-processing / modelling into one object.

**Gridsearch** then lets you test all your assumptions / hyperparameters to find out which combinations generate the best result.

[source](https://www.kaggle.com/evanmiller/pipelines-gridsearch-awesome-ml-pipelines)

In [41]:
from sklearn.model_selection import validation_curve
from sklearn.metrics import r2_score

#from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

# normalize data
X = StandardScaler().fit_transform(X)

# Support Vector Classifier without params
svc = SVC()

k_fold.split(X,y)

# param range for SVC
param_grid = [
 {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
 {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]
# Grid Search using SVC, k_fold, and param_grid
CV = GridSearchCV(estimator=svc, cv=k_fold, param_grid=param_grid, scoring='accuracy')

CV.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
%store CV > CV.txt

#### Validation Curve
This function cross-validates an sklearn estimator (any kind of classifier or regression or pipeline of those things) on the data; but it does this with multiple different hyperparameter settings. Then it plots both the training set and validation set (hold out fold) performance against the parameter values it tried.

This function will borrow results from GridSearch to create the validation curve. I got it from [here](https://matthewbilyeu.com/blog/2019-02-05/validation-curve-plot-from-gridsearchcv-results).

In [ ]:
def plot_grid_search_validation_curve(grid, param_to_vary,
                                      title='Validation Curve', ylim=None,
                                      xlim=None, log=None):
    """Plots train and cross-validation scores from a GridSearchCV instance's
    best params while varying one of those params."""

    df_cv_results = pd.DataFrame(grid.cv_results_)
    train_scores_mean = df_cv_results['mean_train_score']
    valid_scores_mean = df_cv_results['mean_test_score']
    train_scores_std = df_cv_results['std_train_score']
    valid_scores_std = df_cv_results['std_test_score']

    param_cols = [c for c in df_cv_results.columns if c[:6] == 'param_']
    param_ranges = [grid.param_grid[p[6:]] for p in param_cols]
    param_ranges_lengths = [len(pr) for pr in param_ranges]

    train_scores_mean = np.array(train_scores_mean).reshape(*param_ranges_lengths)
    valid_scores_mean = np.array(valid_scores_mean).reshape(*param_ranges_lengths)
    train_scores_std = np.array(train_scores_std).reshape(*param_ranges_lengths)
    valid_scores_std = np.array(valid_scores_std).reshape(*param_ranges_lengths)

    param_to_vary_idx = param_cols.index('param_{}'.format(param_to_vary))

    slices = []
    for idx, param in enumerate(grid.best_params_):
        if (idx == param_to_vary_idx):
            slices.append(slice(None))
            continue
        best_param_val = grid.best_params_[param]
        idx_of_best_param = 0
        if isinstance(param_ranges[idx], np.ndarray):
            idx_of_best_param = param_ranges[idx].tolist().index(best_param_val)
        else:
            idx_of_best_param = param_ranges[idx].index(best_param_val)
        slices.append(idx_of_best_param)

    train_scores_mean = train_scores_mean[tuple(slices)]
    valid_scores_mean = valid_scores_mean[tuple(slices)]
    train_scores_std = train_scores_std[tuple(slices)]
    valid_scores_std = valid_scores_std[tuple(slices)]

    plt.clf()

    plt.title(title)
    plt.xlabel(param_to_vary)
    plt.ylabel('Score')

    if (ylim is None):
        plt.ylim(0.0, 1.1)
    else:
        plt.ylim(*ylim)

    if (not (xlim is None)):
        plt.xlim(*xlim)

    lw = 2

    plot_fn = plt.plot
    if log:
        plot_fn = plt.semilogx

    param_range = param_ranges[param_to_vary_idx]
    if (not isinstance(param_range[0], numbers.Number)):
        param_range = [str(x) for x in param_range]
    plot_fn(param_range, train_scores_mean, label='Training score', color='r',
            lw=lw)
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color='r', lw=lw)
    plot_fn(param_range, valid_scores_mean, label='Cross-validation score',
            color='b', lw=lw)
    plt.fill_between(param_range, valid_scores_mean - valid_scores_std,
                     valid_scores_mean + valid_scores_std, alpha=0.1,
                     color='b', lw=lw)

    plt.legend(loc='lower right')

    plt.show()

In [44]:
# plot validation curve for mean absolute error
print(CV.cv_results_.keys())
print(CV.cv_results_["mean_test_score"])

#plot_grid_search_validation_curve(CV.cv_results_, 'mean_absolute_error', log=True)



# parameters = {}
# parameters['imp__strategy'] = ['mean', 'median', 'most_frequent']
# parameters['feat_select__k'] = [5, 10]
##('feat_select', SelectKBest()),

# # R-squared scoring. 1 is perfect, below 0 is worse than guessing mean of training set y-values.
# train_score, val_score = validation_curve(svm, X, y, param_grid.keys(), param_grid.values(), cv=k_fold, scoring='r2')

# plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
# plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
# plt.legend(loc='best')
# plt.ylim(-.25, 1.25)
# plt.xlabel('degree')
# plt.ylabel('score');
# sns.despine()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_kernel', 'param_gamma', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])
[0.94681794 0.94681794 0.94681794 0.94681794 0.94681794 0.94681794
 0.94681794 0.94681794 0.95088644 0.94681794 0.95408327 0.94681794]


In [12]:
# Someone Else's Code
# i'm using to test unbalanced classes, since the above result uses false assumptions
# (https://towardsdatascience.com/imbalanced-class-sizes-and-classification-models-a-cautionary-tale-part-2-cf371500d1b3)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
models = [{'name': 'logreg','label': 'Logistic Regression',
           'classifier': LogisticRegression(random_state=88),
           'grid': {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}},
          
          {'name': 'knn','label':'K Nearest Neighbors',
           'classifier':KNeighborsClassifier(),
           'grid': {"n_neighbors":np.arange(8)+1}},
          
          {'name': 'dsc','label': 'Descision Tree', 
           'classifier': DecisionTreeClassifier(random_state=88),
           'grid': {"max_depth":np.arange(8)+1}},
          
          {'name': 'rf', 'label': 'Random Forest',
           'classifier': RandomForestClassifier(random_state=88),
           'grid': {'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],
                    'max_depth' : [4,5,6,7,8],'criterion' :['gini', 'entropy']}},
          
          {'name': 'svm_rbf', 'label': 'SVC (RBF)',
           'classifier':SVC(random_state=88),
           'grid': {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}}]

In [15]:
# Now let's make an effort with Random Forest, making sure to design for unbalanced classes
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state=1234)
adasyn = ADASYN(random_state=1234)
grid = {'class__n_estimators': [200, 500],
        'class__max_features': ['auto', 'sqrt', 'log2'],
        'class__max_depth' : [4,5,6,7,8],
        'class__criterion' :['gini', 'entropy']}
pipeline = Pipeline([('sampling', adasyn), ('class', rf)])
grid_cv = GridSearchCV(pipeline, grid, scoring = 'roc_auc', cv = 5)
   
grid_cv.fit(X, y)
grid_cv.best_score_

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

0.8082359666134853

## Current Results
So our best score with Random Forest gave us 88% accuracy, and the SVC with mediocre-scored wines removed gave us almost 95% accuracy.

Obviously we need to do more work and continue to iterate our algorithm approaches, but I feel like this is a good start so far!

## Appendices of Code Attempts
I want to practice writing effective methods, rather than plugging in algorithms agnostic to the nature of the data. As you can see, I'm still drafting solutions to my first dataset, which leaves two other datasets and a research paper as casulties. I don't think this class prepared me for the incohesive jump to this sort of project, but I'm not going to throw everything to the wind just to have words on paper. That's why my focus has been on creating a valid ML solution first before continuing. I hope you understand!!

Thanks

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForest

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# take all our data, and reserve 20% of it for testing 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=12345,
                                                    stratify=y_p)


# Initializing Classifiers
clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=12345)
clf2 = RandomForestClassifier(criterion="entropy")
clf3 = SVC(random_state=12345)

# Building the pipelines
pipe1 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf1)])

pipe2 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf2)])

pipe3 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf3)])


# Setting up the parameter grids
param_grid1 = [{'classifier__penalty': ['l2'],
                'classifier__C': np.power(10., np.arange(-4, 4))}]

param_grid2 = [{'max_depth': np.arange(2,8,2)),
                'classifier__p': [1, 2]}]

param_grid3 = [{'classifier__kernel': ['rbf'],
                'classifier__C': np.power(10., np.arange(-4, 4)),
                'classifier__gamma': np.power(10., np.arange(-5, 0))},
               {'classifier__kernel': ['linear'],
                'classifier__C': np.power(10., np.arange(-4, 4))}]


# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3),
                            (pipe1, pipe2, pipe3),
                            ('Logistic', 'KNN', 'SVM')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=2, # just 2-fold inner loop, i.e. train/test
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

In [ ]:
# (Example)
kf = StratifiedKFold (df['target'], n_folds=10)
mse = []
fold_count = 0
for train, test in kf:
    print("Processing fold %s" % fold_count)
    train_fold = df.ix[train]
    test_fold = df.ix[test]
    
    # find best features
    corr = train_fold.corr()['target'][train_fold.corr()['target'] < 1].abs()
    corr.sort(ascending=False)
    features = corr.index[[0,1]].values
    
    # Get training examples
    train_fold_input  = train_fold[features].values
    train_fold_output = train_fold['target']
    
    # Fit logistic regression
    logreg = LogisticRegression()
    logreg.fit(train_fold_input, train_fold_output)
    
    # Check MSE on test set
    pred = logreg.predict(test_fold[features])
    mse.append(mean_squared_error(test_fold.target, pred))
    
    # Done with the fold
    fold_count += 1

print(DataFrame(mse).mean())